In [1]:
import pandas as pd
import numpy as np
import pickle
from fbprophet import Prophet
import urllib
import urllib.parse
import json
import plotly.graph_objects as go
from urllib.request import urlopen
from weather import *
import dropbox
from dropboxUtils import *
import pandas as pd
import os
import argparse
import contextlib
import datetime
import os
import six
import sys
import time
import unicodedata
from datetime import *
from wwo_hist import retrieve_hist_data
import pytz

In [2]:
weather_api_key = 'd26ec461fcd64cc583b134233201002'
dropbox_api_key = 'wJzLu96m0qIAAAAAAAHPFR8xX-NLBJyqa7QSqSe4Uix8ZzEN6uiYaOyyQLUR6GYJ'
dbx = dropbox.Dropbox(dropbox_api_key)
dbx.users_get_current_account()
print('connected to dropbox')

INFO:dropbox:Request to users/get_current_account


connected to dropbox


In [3]:
df = pd.read_csv(
    'https://www.dropbox.com/s/keafvwlkboedkdm/jghDailyVisits.csv?dl=1')
df.ds = pd.to_datetime(df.ds)
print('jgh daily visits loaded')

jgh daily visits loaded


In [4]:
stat_days_df = pd.read_csv(
    'https://www.dropbox.com/s/hj3byufwtypi8d3/statdays.csv?dl=1')

ramq_stat_days = stat_days_df['RAMQ']
ramq_stat_days = ramq_stat_days.dropna()
jgh_stat_days = stat_days_df['JGH']
jgh_stat_days = jgh_stat_days.dropna()
ramq_stat_days = pd.DataFrame({
    'holiday': 'ramq',
    'ds': ramq_stat_days.to_list(),
    'lower_window': 0,
    'upper_window': 1,
})
jgh_stat_days = pd.DataFrame({
    'holiday': 'jgh',
    'ds': jgh_stat_days.to_list(),
    'lower_window': 0,
    'upper_window': 1,
})
holidays = pd.concat((ramq_stat_days, jgh_stat_days))
print('stat days prepared')

# Get all Montreal Canadiens games in our time frame (2012-present) from the NHL's REST API
hockey_df = pd.read_csv(
    'https://www.dropbox.com/s/g00ih64ndksshwr/montrealHockey.csv?dl=1')
hockey_df.dateTime = pd.to_datetime(hockey_df.dateTime)

# Separate the games by Home or Away
home_hockey_df = hockey_df[hockey_df['homeTeam'] == 'Montreal Canadiens']

away_hockey_df = hockey_df[hockey_df['awayTeam'] == "Montreal Canadiens"]

# Format them as Prophet expects
home_hockey = pd.DataFrame({
    'holiday': 'homeHockey',
    'ds': pd.to_datetime(home_hockey_df['dateTime']),
    'lower_window': 0,
    'upper_window': 1,
})

away_hockey = pd.DataFrame({
    'holiday': 'awayHockey',
    'ds': pd.to_datetime(away_hockey_df['dateTime']),
    'lower_window': 0,
    'upper_window': 1,
})

# Add the hockey games to our stat day data
holidays = pd.concat(
    (home_hockey, away_hockey, ramq_stat_days, jgh_stat_days))
print('hockey games added')

stat days prepared
hockey games added


In [5]:
# Load our weather data from Dropbox
weather_df = pd.read_csv(
    'https://www.dropbox.com/s/688u8aw6k0eqzqb/montrealDailyWeather.csv?dl=1')
weather_df['ds'] = pd.to_datetime(weather_df['ds'])
print('old weather added')

old weather added


In [ ]:
# Fetch whicher days are missing, and 2 days into the future (weather forecast), which is the maximum the WWO API let's us access through the this API
# The Data is saved as a csv file called 'Montreal'
frequency = 24
start_date = (weather_df.ds.max()-timedelta(days=2)
              ).date().strftime("%d-%b-%Y").upper()
end_date = (datetime.now(pytz.utc)).astimezone(
    pytz.timezone('US/Eastern')).date().strftime("%d-%b-%Y").upper()
api_key = weather_api_key
location_list = ['Montreal']
retrieve_hist_data(api_key,
                   location_list,
                   start_date,
                   end_date,
                   frequency,
                   location_label=False,
                   export_csv=True,
                   store_df=False)

missing_weather_df = pd.read_csv('Montreal.csv')
missing_weather_df['date_time'] = pd.to_datetime(
    missing_weather_df['date_time'])
missing_weather_df = missing_weather_df.drop(
    ['moonrise', 'moonset', 'sunrise', 'sunset'], axis=1)
missing_weather_df = missing_weather_df.rename(columns={"date_time": "ds"})
print('new weather fetched')

# Concatenate the old weather data with the missing weather data
final_weather_df = pd.concat([weather_df, missing_weather_df])

In [8]:
final_weather_df.to_csv('montrealDailyWeather.csv', index=False)

upload(dbx, 'montrealDailyWeather.csv', '', '',
       'montrealDailyWeather.csv', overwrite=True)

INFO:dropbox:Request to files/upload


uploaded as b'montrealDailyWeather.csv'


FileMetadata(name='montrealDailyWeather.csv', id='id:HRQT88r_1DgAAAAAAAE2Tw', client_modified=datetime.datetime(2020, 2, 10, 14, 6, 58), server_modified=datetime.datetime(2020, 2, 10, 14, 7), rev='0159e3940c10f7200000001a109f300', size=223259, path_lower='/montrealdailyweather.csv', path_display='/montrealDailyWeather.csv', parent_shared_folder_id=None, media_info=None, symlink_info=None, sharing_info=None, is_downloadable=True, export_info=None, property_groups=None, has_explicit_shared_members=None, content_hash='1bdd93450ee9bedb4d782b9900e07bad0cb48975e965da918cab27065065d785')

In [9]:
regressors = final_weather_df.columns.to_list()
regressors.remove('ds')

df.ds = pd.to_datetime(df.ds)
final_df = pd.merge(df, final_weather_df, on='ds')

In [10]:
final_df

,ds,y,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,uvIndex.1,moon_illumination,DewPointC,...,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
0,2012-05-01,211,12,6,0.0,9.1,1,1,69,7,...,7,15,98,87,1.7,1017,12,7,133,9
1,2012-05-02,201,16,7,0.0,12.4,1,1,76,8,...,10,14,69,86,0.5,1019,16,7,99,9
2,2012-05-03,204,17,10,0.0,9.2,1,1,83,11,...,13,11,90,84,0.3,1017,17,9,120,7
3,2012-05-04,210,17,10,0.0,9.2,1,1,91,12,...,13,11,79,91,3.7,1012,17,5,152,7
4,2012-05-05,180,14,4,0.0,14.5,1,1,98,4,...,8,14,21,70,0.0,1020,14,10,32,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3015,2020-02-07,188,-6,-11,22.5,3.8,1,1,95,-8,...,-15,34,100,94,39.2,992,-6,2,133,24
3016,2020-02-07,188,-6,-10,27.7,3.8,1,1,95,-7,...,-14,35,100,94,42.3,992,-6,2,22,24
3017,2020-02-07,188,-6,-10,22.4,3.8,1,1,95,-8,...,-15,36,99,94,42.8,992,-6,2,135,25
3018,2020-02-08,187,-17,-22,1.0,7.1,1,1,100,-20,...,-28,27,47,89,1.3,1019,-17,9,256,17


In [11]:
print('starting to build model')
# Instantiate our model with our stat day and hockey variables saved in the 'holidays' dataframe
m = Prophet(holidays=holidays, seasonality_mode='multiplicative',
            changepoint_prior_scale=0.5, changepoint_range=0.85)
# Add Prophet's built-in holidays for Canada
m.add_country_holidays(country_name='CA')
# Add our 20 weather variables as regressors
for regressor in regressors:
    m.add_regressor(regressor)
# Fit our model to our data
# m.fit(final_df)
# Fit our model to data from before 2019
m.fit(final_df)
print('finished fitting model')

import pickle
pkl_path = "jgh-prophet-daily.pkl"
with open(pkl_path, "wb") as f:
    # Pickle the 'Prophet' model using the highest protocol available.
    pickle.dump(m, f)

upload(dbx, 'jgh-prophet-daily.pkl', '', '',
       'jgh-prophet-daily.pkl', overwrite=True)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


starting to build model


/anaconda3/envs/prophet/lib/python3.7/site-packages/pystan/misc.py:399: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.

INFO:dropbox:Request to files/upload


finished fitting model
uploaded as b'jgh-prophet-daily.pkl'


FileMetadata(name='jgh-prophet-daily.pkl', id='id:HRQT88r_1DgAAAAAAAE2Uw', client_modified=datetime.datetime(2020, 2, 10, 14, 8, 9), server_modified=datetime.datetime(2020, 2, 10, 14, 8, 12), rev='0159e394509909900000001a109f300', size=816329, path_lower='/jgh-prophet-daily.pkl', path_display='/jgh-prophet-daily.pkl', parent_shared_folder_id=None, media_info=None, symlink_info=None, sharing_info=None, is_downloadable=True, export_info=None, property_groups=None, has_explicit_shared_members=None, content_hash='5df15bea834f0966c1b4a3ea9ef1fe52467231d85c7d57cfebec2105256dcba3')